In [2]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [3]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_left.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        self.do_simulation(action, self.frame_skip)

        observation = self._get_obs()
        reward = 0
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [4]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.046 ,  0.1435, -0.1245,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [5]:
leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [6]:
for i in range(-100, 101, 4):
    for j in range(-100, 101, 4):
        for k in range(-100, 101, 4):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,10):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 9:
                    env_obs = obs
            

            leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [7]:
print(len(leg_dataset))

132651


In [8]:
#Convert left leg list to numpy array
leg_dataset_as_np = np.array(leg_dataset)

In [23]:
print(leg_dataset_as_np[0:4])

observations = (leg_dataset_as_np[:, 0:3][:] + 1)
print(observations)
actions = (leg_dataset_as_np[:, 3:][:])
print(actions)

[[ 0.13726679 -0.15606861 -0.13356711 -1.         -1.         -1.        ]
 [ 0.13965253 -0.15131012 -0.13525802 -1.         -1.         -0.96      ]
 [ 0.14195688 -0.14652161 -0.13658054 -1.         -1.         -0.92      ]
 [ 0.14408437 -0.14162463 -0.13780251 -1.         -1.         -0.88      ]]
[[1.13726679 0.84393139 0.86643289]
 [1.13965253 0.84868988 0.86474198]
 [1.14195688 0.85347839 0.86341946]
 ...
 [1.03507327 1.04806418 1.0711915 ]
 [1.03309455 1.04308471 1.07005505]
 [1.03110725 1.03857524 1.06891366]]
[[0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.04]
 ...
 [1.   1.   0.96]
 [1.   1.   0.98]
 [1.   1.   1.  ]]


In [24]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime
from tensorflow.keras import layers, models


# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture

ik_input = tf.keras.Input(shape=(3,))
x = layers.Dense(128, activation="relu")(ik_input)
x = layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())(x)

ik_output = layers.Dense(3, activation="hard_sigmoid")(x)


model = models.Model(ik_input, ik_output)
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)  # Set the learning rate here
model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['accuracy'])

# Train the model
model.fit(observations, actions, epochs=10, batch_size=2)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)
current_time = datetime.datetime.now()
print("Current Time:", current_time)


Epoch 1/10
 5892/66326 [=>............................] - ETA: 1:17 - loss: 0.2468 - accuracy: 0.3335

KeyboardInterrupt: 

In [21]:
model.save("left_legs_model_plus50", overwrite=True)

INFO:tensorflow:Assets written to: left_legs_model_plus50/assets


INFO:tensorflow:Assets written to: left_legs_model_plus50/assets
